In [62]:
import gspread
import pandas as pd

from speakerAnalysis import speakerAnalysisProject

from importlib import reload
reload(speakerAnalysisProject)

<module 'speakerAnalysis.speakerAnalysisProject' from '/home/seth/analysis/EDI/speakerAnalysis/speakerAnalysisProject.py'>

In [51]:
key = speakerAnalysisProject.readkeyfile('.keys')

In [52]:
# Use gspread api to access spreadsheet
gc = gspread.api_key(key[0])

sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/10vC4mm2HWrlcGIY72Or04t9A3uDUTKhudUAIyPmkF8o/edit?gid=0#gid=0")

print(sh.sheet1.get('A1'))

[['Speaker']]


In [64]:
# Convert spreadsheet into pandas data frame
data = sh.sheet1.get_all_values()
headers = data.pop(0)

df = speakerAnalysisProject.convertGStoPandas(data,headers)
print(df.columns)

Index(['Speaker', 'EDI member', 'Date', 'Type', 'CV?', 'UndergraduateName',
       'UndergraduateCategory', 'GraduateName', 'GraduateCategory',
       'Post-docName', 'Post-docCategory', 'InstitutionName',
       'InstitutionCategory', 'Private/Public', 'Ivy/Other'],
      dtype='object', name=0)


In [65]:
# Fraction of entries from an R1 instituion
notempty = df['InstitutionCategory']!=''
r1 = df['InstitutionCategory'] == 'R1'
print(sum(r1)/sum(notempty))

# Fraction of entries from an R1 undergraduate
notempty = df['Post-docCategory']!=''
r1 = df['Post-docCategory'] == 'R1'
print(sum(r1)/sum(notempty))

# Fraction of entries from an R1 graduate
notempty = df['GraduateCategory']!=''
r1 = df['GraduateCategory'] == 'R1'
print(sum(r1)/sum(notempty))

# Fraction of entries from an R1 undergraduate
notempty = df['UndergraduateCategory']!=''
r1 = df['UndergraduateCategory'] == 'R1'
print(sum(r1)/sum(notempty))

1.0
0.3333333333333333
0.75
0.25
